In [1]:
import numpy as np

In [17]:

X = np.array([[1,2,3],
    [4,5,6], 
    [7,8,9]])

X_centered = X - X.mean(axis=0)

U, s, Vt = np.linalg.svd(X_centered)
print(U, s, Vt)

[[-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]
 [ 0.70710678  0.70710678  0.        ]] [7.34846923e+00 3.62597321e-16 0.00000000e+00] [[ 0.57735027  0.57735027  0.57735027]
 [-0.81649658  0.40824829  0.40824829]
 [ 0.         -0.70710678  0.70710678]]
